In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
import nltk
from nltk.tokenize import word_tokenize

**Load Data**

In [3]:
df=pd.read_csv('train_data.csv')

In [4]:
df.head()

,text,label
0,Here are Thursday's biggest analyst calls: App...,0
1,Buy Las Vegas Sands as travel to Singapore bui...,0
2,"Piper Sandler downgrades DocuSign to sell, cit...",0
3,"Analysts react to Tesla's latest earnings, bre...",0
4,Netflix and its peers are set for a ‘return to...,0


In [5]:
df.shape

(16990, 2)

In [6]:
df['label'].value_counts()

label
2     3545
18    2118
14    1822
9     1557
5      987
16     985
1      837
19     823
7      624
6      524
15     501
17     495
12     487
13     471
4      359
3      321
0      255
8      166
10      69
11      44
Name: count, dtype: int64

**Data Cleansing**

In [7]:
def cleansing(df):
    df_clean=df.str.lower()
    df_clean=[re.sub(r"\d+","",i )for i in df_clean]
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean]
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean]
    return df_clean

In [8]:
df['clean_text']=cleansing(df['text'])

In [9]:
df['clean_text'].iloc[1]

'buy las vegas sands as travel to singapore builds wells fargo says https t co flswicz'

In [10]:
df.head()

,text,label,clean_text
0,Here are Thursday's biggest analyst calls: App...,0,here are thursday s biggest analyst calls appl...
1,Buy Las Vegas Sands as travel to Singapore bui...,0,buy las vegas sands as travel to singapore bui...
2,"Piper Sandler downgrades DocuSign to sell, cit...",0,piper sandler downgrades docusign to sell citi...
3,"Analysts react to Tesla's latest earnings, bre...",0,analysts react to tesla s latest earnings brea...
4,Netflix and its peers are set for a ‘return to...,0,netflix and its peers are set for a return to ...


In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

**Split train test**

In [12]:
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['label'], test_size = 0.2, random_state = 42,stratify=df['label'])

In [13]:
x_train.head()

5922     it s ecb rate decision day here s what to expe...
13498    twitter users were quick to spot liz truss see...
4517     jetblue announces webcast of second quarter ea...
16161     calm cal maine foods stock ticks higher on re...
1745     tower semiconductor and cadence expand collabo...
Name: clean_text, dtype: object

**Tokenize the words in the train and test data**

In [14]:
word_token_train = [word_tokenize(i) for i in x_train]
word_token_test = [word_tokenize(i) for i in x_test]

In [15]:
word_token_train[0:1]

[['it',
  's',
  'ecb',
  'rate',
  'decision',
  'day',
  'here',
  's',
  'what',
  'to',
  'expect',
  'via',
  'weberalexander',
  'amp',
  'carolynnlook',
  'https',
  't',
  'co',
  'isqgdue']]

**Remove Stop word**

In [16]:
from nltk.corpus import stopwords
nltk.download('stopwords')
# Get a list of stop words in the Indonesian language
stop_words = set(stopwords.words('english'))

# Display the top 20 stop words
list(stop_words)[:20]

[nltk_data] Downloading package stopwords to /home/andrew/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['more',
 've',
 'for',
 "wouldn't",
 'just',
 'they',
 'until',
 'wasn',
 're',
 'my',
 "she'll",
 "he's",
 "you're",
 'too',
 "aren't",
 'once',
 'above',
 'after',
 'so',
 'herself']

In [17]:
# Remove stopwords from each sublist in word_token_train and test
filtered_tokens_train = [[word for word in sublist if word not in stop_words] for sublist in word_token_train]
filtered_tokens_test = [[word for word in sublist if word not in stop_words] for sublist in word_token_test]


In [18]:
filtered_tokens_train[0:2]

[['ecb',
  'rate',
  'decision',
  'day',
  'expect',
  'via',
  'weberalexander',
  'amp',
  'carolynnlook',
  'https',
  'co',
  'isqgdue'],
 ['twitter',
  'users',
  'quick',
  'spot',
  'liz',
  'truss',
  'seemingly',
  'recreating',
  'outfit',
  'margaret',
  'thatcher',
  'appearance',
  'channel',
  'tory',
  'leadership',
  'debate',
  'https',
  'co',
  'vsiioegrz']]

**Text representation and Featurev**

In [19]:
import gensim
from gensim.models import Word2Vec

# Skipgram
model_skipgram = gensim.models.Word2Vec(filtered_tokens_train, min_count = 3, vector_size = 50, window = 5, sg=1)

In [20]:
#average word2vec for each word
def text_to_vector(text):
    word_vectors = [model_skipgram.wv[word] for word in text if word in model_skipgram.wv]
    return np.mean(word_vectors, axis=1) if word_vectors else np.zeros(3)

In [21]:
# Transforming tokens in train and test into vectors using Skipgram model
x_train_vectors_skipgram = [text_to_vector(text) for text in filtered_tokens_train]
x_test_vectors_skipgram = [text_to_vector(text) for text in filtered_tokens_test]

In [22]:
x_train_vectors_skipgram=[arr.tolist() for arr in x_train_vectors_skipgram]
x_test_vectors_skipgram=[arr.tolist() for arr in x_test_vectors_skipgram]

In [23]:
#padding the token
maxlen= max(len(s) for s in filtered_tokens_train)
padded_train = [seq + [0] * (maxlen - len(seq)) for seq in x_train_vectors_skipgram]
padded_test=[seq + [0] * (maxlen - len(seq)) for seq in x_test_vectors_skipgram]

# Convert to a NumPy array if needed
padded_train = np.array(padded_train, dtype=np.float32)
padded_test = np.array(padded_test, dtype=np.float32)

print(padded_train)

[[ 3.3851415e-02  1.3072202e-01  2.8924594e-02 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 3.3170786e-02 -7.0760451e-04  4.5092646e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 9.9307748e-05 -6.1654657e-02  1.8589402e-02 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 ...
 [-1.7860703e-03 -5.7566762e-03  4.2956327e-03 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [-2.9950282e-03  1.1225151e-02  1.2665372e-02 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 3.3851415e-02  7.0008308e-02  3.3851415e-02 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


In [24]:
from sklearn import svm
svm_class = svm.LinearSVC( random_state=42)
svm_class.fit(padded_train, y_train)

,penalty,'l2'
,loss,'squared_hinge'
,dual,'auto'
,tol,0.0001
,C,1.0
,multi_class,'ovr'
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,verbose,0
,random_state,42


**Testing**

In [25]:
test_svm_class=svm_class.predict(padded_test)

In [26]:
print('\nClassification Report\n')
print(classification_report(y_test, test_svm_class, target_names=['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19']))


Classification Report

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        51
           1       0.34      0.07      0.12       167
           2       0.27      0.83      0.41       709
           3       0.00      0.00      0.00        64
           4       0.82      0.43      0.56        72
           5       0.23      0.20      0.22       198
           6       0.00      0.00      0.00       105
           7       0.00      0.00      0.00       125
           8       0.00      0.00      0.00        33
           9       0.00      0.00      0.00       311
          10       0.00      0.00      0.00        14
          11       0.00      0.00      0.00         9
          12       0.00      0.00      0.00        97
          13       0.00      0.00      0.00        94
          14       0.24      0.32      0.27       364
          15       0.00      0.00      0.00       100
          16       0.00      0.00      0.00       197
   

/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/home/andrew/miniconda3/envs/i_guess_thats_all-env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1706: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beha